In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import scipy.signal as signal

In [2]:
seed_metadata = pd.read_csv("Optimized-Seeds/seed-metadata.csv")

In [3]:
display(seed_metadata)

,Seed #,Mask Size,Fill Fraction,Magnification,Hole Size Limit,Cross-Corr Weight,Sensitivity Weight,Date Started,Date Finished,Final # Iterations
0,81,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"52,132"
1,82,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"65,207"
2,83,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"59,431"
3,84,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"29,836"
4,85,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"57,368"
...,...,...,...,...,...,...,...,...,...,...
85,156,46,0.7,4,80,1,2,7/21/23,7/21/23,"78,204"
86,157,46,0.7,4,80,1,2,7/22/23,7/23/23,"86,457"
87,158,46,0.7,4,80,1,2,7/22/23,7/23/23,"104,833"
88,159,46,0.7,4,80,1,2,7/23/23,7/24/23,"75,469"


In [4]:
test = ['Optimized-Seeds/'+file for file in os.listdir('Optimized-Seeds') if file != 'seed-metadata.csv']

In [5]:
os.listdir(test[1])

['Initial_mask.txt', 'final_data.npy', 'final_mask.txt', 'Plots']

In [6]:
test[1]

'Optimized-Seeds/Seed-103'

In [7]:
seed_metadata

,Seed #,Mask Size,Fill Fraction,Magnification,Hole Size Limit,Cross-Corr Weight,Sensitivity Weight,Date Started,Date Finished,Final # Iterations
0,81,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"52,132"
1,82,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"65,207"
2,83,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"59,431"
3,84,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"29,836"
4,85,46,0.3,4,FALSE,1,1,8/10/23,8/10/23,"57,368"
...,...,...,...,...,...,...,...,...,...,...
85,156,46,0.7,4,80,1,2,7/21/23,7/21/23,"78,204"
86,157,46,0.7,4,80,1,2,7/22/23,7/23/23,"86,457"
87,158,46,0.7,4,80,1,2,7/22/23,7/23/23,"104,833"
88,159,46,0.7,4,80,1,2,7/23/23,7/24/23,"75,469"


In [8]:
# Initialize a list to store the calculated mask arrays
mask_arrays = []

# Iterate through each row in the DataFrame
for index, row in seed_metadata.iterrows():
    seed_number = row['Seed #']
    fill_frac = row['Fill Fraction']
    magnification = row ['Magnification']
    mask_size = row['Mask Size']
    corr_weight = row['Cross-Corr Weight']
    sensitivity_weight = row['Sensitivity Weight']
    
    # Define the directory path containing the files
    directory_path = "/Users/owner/Downloads/test[1]"

    # Read the initial mask from initial_mask.txt  --- Initial
    initial_mask_file_path = 'Optimized-Seeds/Seed-{}/Initial_mask.txt'.format(seed_number)
    initial_mask = np.loadtxt(initial_mask_file_path)
    
    # Cross correlation metric
    F_matrix = np.array([(signal.correlate2d(initial_mask, initial_mask[m:m+math.floor(37/magnification), n:n+math.floor(37/magnification)], mode='valid')/(signal.correlate2d(initial_mask, initial_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2)).reshape(signal.correlate2d(initial_mask, initial_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2, ) for m in range(0, mask_size-math.floor(37/magnification)+1) for n in range(0, mask_size-math.floor(37/magnification)+1)])
    Q1 = (1/math.floor(37/magnification))*np.sum((np.diag(F_matrix)-fill_frac)**4)
    np.fill_diagonal(F_matrix, fill_frac**2)
    Q2 = (1/(math.floor(37/magnification)**2-math.floor(37/magnification)))*np.sum((F_matrix-fill_frac**2)**4)


    # "Sensitivity" metric
    sens_mask = np.copy(initial_mask)
    sens_mask[sens_mask == 0] = 0.05
    sensitivity_matrix = signal.correlate2d(sens_mask, np.ones((math.floor(37/magnification), math.floor(37/magnification))), mode='valid')
    cross_corr_metricI = Q1+Q2
    sensitivity_metricI = np.var(sensitivity_matrix)
    final_metricI = corr_weight*(cross_corr_metricI) + sensitivity_weight*(sensitivity_metricI)
    
    
    # Read the initial mask from initial_mask.txt -- Final
    final_mask_file_path = 'Optimized-Seeds/Seed-{}/final_mask.txt'.format(seed_number)
    final_mask = np.loadtxt(final_mask_file_path)
    
    # Cross correlation metric
    F_matrix = np.array([(signal.correlate2d(final_mask, final_mask[m:m+math.floor(37/magnification), n:n+math.floor(37/magnification)], mode='valid')/(signal.correlate2d(final_mask, final_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2)).reshape(signal.correlate2d(final_mask, final_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2, ) for m in range(0, mask_size-math.floor(37/magnification)+1) for n in range(0, mask_size-math.floor(37/magnification)+1)])
    Q1 = (1/math.floor(37/magnification))*np.sum((np.diag(F_matrix)-fill_frac)**4)
    np.fill_diagonal(F_matrix, fill_frac**2)
    Q2 = (1/(math.floor(37/magnification)**2-math.floor(37/magnification)))*np.sum((F_matrix-fill_frac**2)**4)


    # "Sensitivity" metric
    sens_mask = np.copy(final_mask)
    sens_mask[sens_mask == 0] = 0.05
    sensitivity_matrix = signal.correlate2d(sens_mask, np.ones((math.floor(37/magnification), math.floor(37/magnification))), mode='valid')
    cross_corr_metricF = Q1+Q2
    sensitivity_metricF = np.var(sensitivity_matrix)
    final_metricF = corr_weight*(cross_corr_metricF) + sensitivity_weight*(sensitivity_metricF)
    
    
#magnification of 3
    magnification = 3
     # Cross correlation metric
    F_matrix = np.array([(signal.correlate2d(final_mask, final_mask[m:m+math.floor(37/magnification), n:n+math.floor(37/magnification)], mode='valid')/(signal.correlate2d(final_mask, final_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2)).reshape(signal.correlate2d(final_mask, final_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2, ) for m in range(0, mask_size-math.floor(37/magnification)+1) for n in range(0, mask_size-math.floor(37/magnification)+1)])
    Q1 = (1/math.floor(37/magnification))*np.sum((np.diag(F_matrix)-fill_frac)**4)
    np.fill_diagonal(F_matrix, fill_frac**2)
    Q2 = (1/(math.floor(37/magnification)**2-math.floor(37/magnification)))*np.sum((F_matrix-fill_frac**2)**4)


    # "Sensitivity" metric
    sens_mask = np.copy(final_mask)
    sens_mask[sens_mask == 0] = 0.05
    sensitivity_matrix = signal.correlate2d(sens_mask, np.ones((math.floor(37/magnification), math.floor(37/magnification))), mode='valid')
    cross_corr_metric3 = Q1+Q2
    sensitivity_metric3 = np.var(sensitivity_matrix)
    final_metric3 = corr_weight*(cross_corr_metric3) + sensitivity_weight*(sensitivity_metric3)
    
    
#magnification of 2
    magnification = 2
     # Cross correlation metric
    F_matrix = np.array([(signal.correlate2d(final_mask, final_mask[m:m+math.floor(37/magnification), n:n+math.floor(37/magnification)], mode='valid')/(signal.correlate2d(final_mask, final_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2)).reshape(signal.correlate2d(final_mask, final_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2, ) for m in range(0, mask_size-math.floor(37/magnification)+1) for n in range(0, mask_size-math.floor(37/magnification)+1)])
    Q1 = (1/math.floor(37/magnification))*np.sum((np.diag(F_matrix)-fill_frac)**4)
    np.fill_diagonal(F_matrix, fill_frac**2)
    Q2 = (1/(math.floor(37/magnification)**2-math.floor(37/magnification)))*np.sum((F_matrix-fill_frac**2)**4)


    # "Sensitivity" metric
    sens_mask = np.copy(final_mask)
    sens_mask[sens_mask == 0] = 0.05
    sensitivity_matrix = signal.correlate2d(sens_mask, np.ones((math.floor(37/magnification), math.floor(37/magnification))), mode='valid')
    cross_corr_metric2 = Q1+Q2
    sensitivity_metric2 = np.var(sensitivity_matrix)
    final_metric2 = corr_weight*(cross_corr_metric2) + sensitivity_weight*(sensitivity_metric2)
    
    
    
    
    # Append the mask array to the list
    mask_arrays.append((seed_number, cross_corr_metricI, sensitivity_metricI, final_metricI, cross_corr_metricF, sensitivity_metricF, final_metricF, cross_corr_metric3, sensitivity_metric3, final_metric3, cross_corr_metric2, sensitivity_metric2, final_metric2))

 #Print the mask arrays
# for seed_number, cross_corr_metricI, sensitivity_metricI, final_metricI, cross_corr_metricF, sensitivity_metricF, final_metricF in mask_arrays:
#     print(f"Seed #{seed_number}: \ncross_corr_metricI = {cross_corr_metricI} cross_corr_metricF = {cross_corr_metricF} \nsensitivity_metricI = {sensitivity_metricI} sensitivy_metricF = {sensitivity_metricF} \nInitial Final Metric = {final_metricI}: Final Final Metric {final_metricF}")
#     print('--------------------------------')
    
    
# Add the results as an extra column to the DataFrame
seed_metadata['cross_corr_metricI'] = [values[1] for values in mask_arrays]
seed_metadata['cross_corr_metricF'] = [values[4] for values in mask_arrays]
seed_metadata['cross_corr_metric3'] = [values[7] for values in mask_arrays]
seed_metadata['cross_corr_metric2'] = [values[10] for values in mask_arrays]

seed_metadata['sensitivity_metricI'] = [values[2] for values in mask_arrays]
seed_metadata['sensitivity_metricF'] = [values[5] for values in mask_arrays]
seed_metadata['sensitivity_metric3'] = [values[8] for values in mask_arrays]
seed_metadata['sensitivity_metric2'] = [values[11] for values in mask_arrays]

seed_metadata['final_metricI'] = [values[3] for values in mask_arrays]
seed_metadata['final_metricF'] = [values[6] for values in mask_arrays]
seed_metadata['final_metric3'] = [values[9] for values in mask_arrays]
seed_metadata['final_metric2'] = [values[12] for values in mask_arrays]




# Save the DataFrame back to the CSV file
output_csv_path = "/Users/owner/Downloads/seed-metadata-with-results.csv"
seed_metadata.to_csv(output_csv_path, index=False)

print("Results added to the CSV file.")

Results added to the CSV file.


In [9]:
[values[1] for values in mask_arrays]

[1.2093495965972214,
 1.2109781374458555,
 1.1965623423319531,
 1.2218229445468443,
 1.2076309773431901,
 4.34562727635086,
 4.404939405158013,
 4.376303261840736,
 4.3734421845531,
 4.411169406828694,
 14.157749221279644,
 14.260948231476634,
 14.140574944787566,
 14.12198417413107,
 14.249528481650298,
 39.765073856582966,
 39.840037753320004,
 39.68583674943156,
 39.641538899039986,
 39.66165699686172,
 98.20314298202877,
 97.86368811243393,
 98.41399268840107,
 97.99352175760963,
 98.22401501904824,
 221.25343512988158,
 221.81651564859362,
 220.5638774795644,
 221.21710530293976,
 221.16084887200046,
 458.33186318496763,
 458.6611897728771,
 459.2848494851952,
 458.859099816259,
 457.74765991307055,
 888.2587353836468,
 891.4191780482531,
 889.8187442755943,
 891.4873961912141,
 890.7560518306583,
 1638.244802872589,
 1637.3856964560568,
 1638.1240850117495,
 1635.3822273574422,
 1635.2015088982155,
 1.2002237080913087,
 1.2190244299869457,
 1.2338308705186891,
 1.207440072115735,

initial_mask.shape

In [10]:
# Cross correlation metric
F_matrix = np.array([(signal.correlate2d(initial_mask, initial_mask[m:m+math.floor(37/magnification), n:n+math.floor(37/magnification)], mode='valid')/(signal.correlate2d(initial_mask, initial_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2)).reshape(signal.correlate2d(initial_mask, initial_mask[0:math.floor(37/magnification), 0:math.floor(37/magnification)], mode='valid').shape[0]**2, ) for m in range(0, mask_size-math.floor(37/magnification)+1) for n in range(0, mask_size-math.floor(37/magnification)+1)])
Q1 = (1/math.floor(37/magnification))*np.sum((np.diag(F_matrix)-fill_frac)**4)
np.fill_diagonal(F_matrix, fill_frac**2)
Q2 = (1/(math.floor(37/magnification)**2-math.floor(37/magnification)))*np.sum((F_matrix-fill_frac**2)**4)


# "Sensitivity" metric
sens_mask = np.copy(initial_mask)
sens_mask[sens_mask == 0] = 0.05
sensitivity_matrix = signal.correlate2d(sens_mask, np.ones((math.floor(37/magnification), math.floor(37/magnification))), mode='valid')
cross_corr_metric = Q1+Q2
sensitivity_metric = np.var(sensitivity_matrix)
final_metric = corr_weight*(cross_corr_metric) + sensitivity_weight*(sensitivity_metric)

In [ ]:
cross_corr_metric

106.21925332137094

In [ ]:
final_metric

139.7559007217676

In [ ]:
sensitivity_metric

16.768323700198334